In [7]:
import numpy.random as rd
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation as ani
import matplotlib.cm as cm
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)

from tabulate import tabulate
from time import time

import pystan
from pystan import StanModel

### data

In [6]:
data = np.array([[28,15],[8,10],[-3,16],[7,11],[-1,9],[1,11],[18,10],[12,18]])